#### Spark UI

how to access the spark UI

YARN Resource manager and Spark UI

In [16]:
from pyspark.sql import SparkSession

In [17]:
spark = SparkSession. \
builder. \
appName("caching session demo3"). \
config("spark.sql.warehouse.dir","/user/itv020649/warehouse"). \
master('yarn'). \
getOrCreate()

In [22]:
spark

#### Cashe and Persist

spark is already in memory than why cache()?

df1 = read a order file from disk -- if i fee this df1 will be reused again and again we can cache it.

df2 = df1.xfr1  df3 = df2.xfr2

df3.show() --- all stages will run from begning from read.

this is solved in cache() (it is lazy)

should not cache() large DF or it can make issue

RDD - cache in memory

DF or sql table - it cache in memory and disk

10 GB file- 128 block size- 80 blocks-not enough memrory (only 50 are in memory)-remaining 50 in disk- this disk part is skipped in RDD

Disk has 2 parts - worker node-hdfs / local disk(little faster)

persist- memory and disk as option

cache- memory+disk by default

### Cache praticals

In [19]:
orders_schema = 'order_id long,order_date date,cust_id long,order_status string'

In [20]:
raw_df = spark.read \
.format("csv") \
.schema(orders_schema) \
.load("/public/trendytech/orders/orders_1gb.csv")

In [23]:
raw_df.show()

+--------+----------+-------+---------------+
|order_id|order_date|cust_id|   order_status|
+--------+----------+-------+---------------+
|       1|2013-07-25|  11599|         CLOSED|
|       2|2013-07-25|    256|PENDING_PAYMENT|
|       3|2013-07-25|  12111|       COMPLETE|
|       4|2013-07-25|   8827|         CLOSED|
|       5|2013-07-25|  11318|       COMPLETE|
|       6|2013-07-25|   7130|       COMPLETE|
|       7|2013-07-25|   4530|       COMPLETE|
|       8|2013-07-25|   2911|     PROCESSING|
|       9|2013-07-25|   5657|PENDING_PAYMENT|
|      10|2013-07-25|   5648|PENDING_PAYMENT|
|      11|2013-07-25|    918| PAYMENT_REVIEW|
|      12|2013-07-25|   1837|         CLOSED|
|      13|2013-07-25|   9149|PENDING_PAYMENT|
|      14|2013-07-25|   9842|     PROCESSING|
|      15|2013-07-25|   2568|       COMPLETE|
|      16|2013-07-25|   7276|PENDING_PAYMENT|
|      17|2013-07-25|   2667|       COMPLETE|
|      18|2013-07-25|   1205|         CLOSED|
|      19|2013-07-25|   9488|PENDI

In [7]:
raw_df.printSchema()

root
 |-- order_id: long (nullable = true)
 |-- order_date: date (nullable = true)
 |-- cust_id: long (nullable = true)
 |-- order_status: string (nullable = true)



In [8]:
raw_df.count()

25831125

1.1gb

9 blocks in hdfs

count

9 partition in your data frame

9 task
task1 -> 1000
task2 -> 2000
task3 -> 1500

1 from each executor and sum

In [9]:
order_df_cached = raw_df.cache()

In [10]:
order_df_cached.show()

KeyboardInterrupt: 

for show it only cached 1st partition 11% same for head-- check storage

for tail it will cache will be cached

In [ ]:
order_df_cached.tail(10)

Memory Deserilized- object form like formal data,takes more space

serialized - binary format(more optimizedterm of space, but convertion it takes extra cpu cycle)

in-memory-> deserilized
disk-> serialized

caching:
in-memory-> deserilized/serialized
disk-> serialized

In [ ]:
order_df_cached.count()
#it will deal with all records so all partition will be cached

dynamic resource allocation is enabled, min two executor and max 10

id application is ideal the extra executor will be removed but not less then min

node_loacl -> normal/disk

processed local -> cache

In [ ]:
order_df_cached.unpersist()

### More on CACHE

In [7]:
raw_df.cache()

order_id,order_date,cust_id,order_status
1,2013-07-25,11599,CLOSED
2,2013-07-25,256,PENDING_PAYMENT
3,2013-07-25,12111,COMPLETE
4,2013-07-25,8827,CLOSED
5,2013-07-25,11318,COMPLETE
6,2013-07-25,7130,COMPLETE
7,2013-07-25,4530,COMPLETE
8,2013-07-25,2911,PROCESSING
9,2013-07-25,5657,PENDING_PAYMENT
10,2013-07-25,5648,PENDING_PAYMENT


In [8]:
filtered_results = raw_df.filter("cust_id == 1000000")


In [9]:
filtered_results.show()
# it will cache everything

+--------+----------+-------+------------+
|order_id|order_date|cust_id|order_status|
+--------+----------+-------+------------+
+--------+----------+-------+------------+



In [10]:
raw_df.select("order_id","order_status").filter("order_status=='CLOSED'").cache()

order_id,order_status
1,CLOSED
4,CLOSED
12,CLOSED
18,CLOSED
24,CLOSED
25,CLOSED
37,CLOSED
51,CLOSED
57,CLOSED
61,CLOSED


In [11]:
raw_df.select("order_id","order_status").filter("order_status=='CLOSED'").cache().count()
#count will cache everything
#if rerun it will hit cache

2833500

In [14]:
raw_df.filter("order_status=='CLOSED'").select("order_id","order_status").cache().count()
#even the query is samemoving filters ahead so it doesnt hit chache
#cache only checks analysised plan not optimised and if it matches its hit cache


2833500

In [ ]:
raw_df.select("order_id").filter("order_status=='CLOSED'").cache().count()
#it will hit cache

In [ ]:
raw_df.select("order_id").filter("order_status=='CLOSED'").cache().count()
#it will not hit cache

### Cache - InMemory Table Cache | Node Local & Process Local

In [24]:
raw_df.count()

Py4JJavaError: An error occurred while calling o187.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: ResultStage 4 (count at NativeMethodAccessorImpl.java:0) has failed the maximum allowable number of times: 4. Most recent failure reason: org.apache.spark.shuffle.FetchFailedException: java.lang.IllegalArgumentException: Unknown message type: 10 	at org.apache.spark.network.shuffle.protocol.BlockTransferMessage$Decoder.fromByteBuffer(BlockTransferMessage.java:71) 	at org.apache.spark.network.shuffle.ExternalShuffleBlockHandler.receive(ExternalShuffleBlockHandler.java:80) 	at org.apache.spark.network.server.TransportRequestHandler.processRpcRequest(TransportRequestHandler.java:180) 	at org.apache.spark.network.server.TransportRequestHandler.handle(TransportRequestHandler.java:103) 	at org.apache.spark.network.server.TransportChannelHandler.channelRead(TransportChannelHandler.java:118) 	at org.spark_project.io.netty.channel.AbstractChannelHandlerContext.invokeCh